In [ ]:
!pip -q install caer
!pip -q install streamlit
!pip -q install pyngrok
!pip install ipykernel>=5.1.2
!pip install pydeck
!pip install streamlit-webrtc

In [ ]:
from google.colab import drive
drive.mount("/content/drive",force_remount=True)

Mounted at /content/drive


In [ ]:
%%writefile app.py
import numpy as np
import cv2
import tensorflow as tf
import streamlit as st
from PIL import Image
from streamlit_webrtc import VideoProcessorBase,webrtc_streamer,RTCConfiguration,WebRtcMode
import cv2

haar_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')


class VideoProcessor(VideoProcessorBase):
    def __init__(self):
        self.i = 0
    def transform(self,frame):
      image = frame.to_ndarray(format="bgr24")
      gray = cv2.cvtColor(image,cv2.COLOR_BGR2GRAY)
      faces_roi = haar_cascade.detectMultiScale(gray,1.3,5)
      i = self.i+1
      for (x,y,w,h) in faces_roi:
        cv2.rectangle(image,(x,y),(x+w,y+h),(255,0,0),3)
      return image


@st.cache(allow_output_mutation=True)
def load_model():
    model = tf.keras.models.load_model("/content/drive/MyDrive/model.h5")
    return model
model = load_model()

def pil_to_cv2_image(image):
    opencv_array = np.array(image)
    cv2.imwrite('out.jpg', cv2.cvtColor(opencv_array, cv2.COLOR_RGB2BGR))
    opencv_image = cv2.imread('out.jpg')
    return opencv_image

def detect_image(image):
    img = pil_to_cv2_image(image)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    faces_roi = haar_cascade.detectMultiScale(gray, scaleFactor=1.5, minNeighbors=3)
    for (x, y, w, h) in faces_roi:
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        cv2.rectangle(img, (x, y), (x + w, y + h), (0, 0, 255), 2)
    return img

def welcome():
    st.title('Detection et classifiction des images en utilisant streamlit')
    st.subheader('Une appication simple qui vout permet de choisir entre trois options dispobible au niveau du sidebar')

def image_detection():
    st.header("Détection sur une image chargée")
    html_body = """<body style="background-color:red;"></body>"""
    st.markdown(html_body, unsafe_allow_html=True)
    html_temp = """
            <body style="background-color:red;">
            <div style="background-color:red;padding:10px;">
            <h2 style="color:white;text-align:center;">Détection De Visages Avec HaarCascade</h2>
            </div>
            </body>
            """
    st.markdown(html_temp, unsafe_allow_html=True)
    image_file = st.file_uploader("Charger Une Image", type=['jpg', 'png', 'jpeg'], key=0)
    if image_file is not None:
        our_image = Image.open(image_file)
        st.text("Original Image")
        st.image(our_image)
    if st.button("Detect", key=20):
        result_image = detect_image(our_image)
        st.text("Results")
        st.image(result_image)

def live_detection():
    st.header("Détection De Visage En Direct")
    html_temp = """
            <body>
            <div style="padding:15px;text-align:center;margin:25px;">
            <h4 style = "color:black;text-align:center;">Utilisation De La Webcam</h4>
            </div>
            </body>
            """
    st.markdown(html_temp, unsafe_allow_html=True)
    RTC_CONFIGURATION = RTCConfiguration({"iceServers":[{"urls": ["stun:stun.l.google.com:19302"]}]})
    webrtc_streamer(key="face_detection",video_processor_factory=VideoProcessor,mode=WebRtcMode.SENDRECV,rtc_configuration=RTC_CONFIGURATION)

def simpsons_recognition():
    class_names = ['bart_simpson', 'charles_montgomery_burns', 'homer_simpson', 'krusty_the_clown',
                   'lisa_simpson', 'marge_simpson', 'milhouse_van_houten', 'moe_szyslak', 'ned_flanders',
                   'principal_skinner']
    st.header("Reconnaissance des personnages des Simpsons")
    html_body = """<body style="background-color:red;"></body>"""
    st.markdown(html_body, unsafe_allow_html=True)
    html_temp = """
            <body style="background-color:red;">
            <div style="background-color:red;padding:10px;">
            <h2 style="color:white;text-align:center;">Détection basée sur un modèle de Deep Learning</h2>
            </div>
            </body>
            """
    st.markdown(html_temp, unsafe_allow_html=True)
    image_file_2 = st.file_uploader("Charger Une Image", type=['jpg', 'png', 'jpeg'], key=1)
    if image_file_2 is not None:
        our_image_1 = Image.open(image_file_2)
        st.text("Original Image")
        st.image(our_image_1)
    if st.button("Recognise"):
        if image_file_2 is not None:
            opencv_image = pil_to_cv2_image(our_image_1)
            opencv_image = cv2.resize(opencv_image, (80, 80), interpolation=cv2.INTER_AREA)
            opencv_image = np.array(opencv_image)
            opencv_image = opencv_image.astype('float32')
            opencv_image = np.expand_dims(opencv_image, axis=0)
            opencv_image /= 255
            result = class_names[np.argmax(model.predict(opencv_image))]
            st.text("Results")
            st.text("Ce personnage est " + result)

selected_box = st.sidebar.selectbox(
        'Choisissez une des options suivantes',
        ('Bienvenue', 'Detection en direct', 'Detection sur une image uploadée',
         'Reconnaissance des personnages de Simpsons'))
        
if selected_box == 'Bienvenue':
  welcome()
if selected_box == 'Detection en direct':
  live_detection()
if selected_box == 'Detection sur une image uploadée':
  image_detection()
if selected_box == 'Reconnaissance des personnages de Simpsons':
  simpsons_recognition()


Overwriting app.py


In [ ]:
!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip

--2021-10-06 11:03:59--  https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
Resolving bin.equinox.io (bin.equinox.io)... 52.202.168.65, 18.205.222.128, 54.161.241.46, ...
Connecting to bin.equinox.io (bin.equinox.io)|52.202.168.65|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 13832437 (13M) [application/octet-stream]
Saving to: ‘ngrok-stable-linux-amd64.zip.2’

ngrok-stable-linux- 100%[===================>]  13.19M  31.1MB/s    in 0.4s    

2021-10-06 11:04:00 (31.1 MB/s) - ‘ngrok-stable-linux-amd64.zip.2’ saved [13832437/13832437]



In [ ]:
!unzip ngrok-stable-linux-amd64.zip

Archive:  ngrok-stable-linux-amd64.zip
replace ngrok? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: ngrok                   


In [ ]:
get_ipython().system_raw('./ngrok http 8501 &')

In [ ]:
!curl -s http://localhost:4040/api/tunnels | python3 -c \
    'import sys, json; print("Execute the next cell and the go to the following URL: " +json.load(sys.stdin)["tunnels"][0]["public_url"])'

Execute the next cell and the go to the following URL: https://7b20-35-194-14-119.ngrok.io


In [ ]:
!streamlit run app.py


  You can now view your Streamlit app in your browser.

  Network URL: http://172.28.0.2:8501
  External URL: http://35.194.14.119:8501

2021-10-06 11:04:14.959851: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2021-10-06 11:04:14.959912: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (3be18cc356a7): /proc/driver/nvidia/version does not exist
2021-10-06 11:04:15.681 Error in loading the saved optimizer state. As a result, your model is starting with a freshly initialized optimizer.
/usr/local/lib/python3.7/dist-packages/aiortc/rtcdtlstransport.py:211: CryptographyDeprecationWarning: This version of cryptography contains a temporary pyOpenSSL fallback path. Upgrade pyOpenSSL now.
  _openssl_assert(lib.SSL_CTX_use_certificate(ctx, self._cert._x509) == 1)  # type: ignore
2021-10-06 11:04:27.316 Connection(0) Remote candidate "01c2560